<img src="../qbies.png" style="width: 100px;padding-right:5px;padding-top:1px;padding-left:5px;" align="left"/>

    
<p style="padding-left:125px;padding-top: 30px";><font size="+2"><b> Practical Guidance - Lists
    </b></font><p>
 

## Creating a list - `enlist` vs `(),`
Guaranteeing list structures - differences and when to use which.

We saw in the main content that we can create a list by appending an empty list to an atom: 

In [1]:
a:23
(),a
type (),a

,23


7h


We can also create a list from an atom by using the keyword [enlist](https://code.kx.com/q4m3/A_Built-in_Functions/#a26-enlist)

In [2]:
enlist a
type enlist a

,23


7h


There is a difference between using the `enlist` versus `(),` syntax, as enlisting will **always** create a new level of nesting. We can see this by comparing the type outputs and how they differ between the two techniques when we pass a list rather than an atom.

In [3]:
a: 1 2; 
enlist a 
(),a

type enlist a    //enlisting in this case creates a list of lists
type (),a        //joining an empty list here doesn't create a new level of nesting

1 2


1 2


0h


7h


<img src="../qbies.png" style="width: 50px;padding-right:5px;padding-top:15px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Enlisting <b>always</b> creates a new level of nesting while joining an empty list to the beginning of an item will just ensure that the final result is a list. Being aware of the result we are trying to achieve is key when deciding between the two techniques. If in doubt, use the empty list append to avoid unexpected structures! </i></p>

## Dealing with null values

When working with data we need to be able to deal effectively with null values - typically this will mean we need to replace nulls with some non-null values, or filter out the nulls from our data. 

### Fill operator 
The first operator to discuss is denoted by the primitive [`^`](https://code.kx.com/q/ref/fill/), known as fill or coalesce.  The
simplest use case is where the fill operator replaces nulls in a list with a provided value.

In [4]:
show g:1 2 0N 0N 3 4 0N
0^g                          //replace any nulls with 0

1 2 0N 0N 3 4 0N


1 2 0 0 3 4 0


Another commonly used value to replace null values is the average value from the rest of the list. It's easy to do this in kdb+/q as the `avg` keyword ignores null values. 

In [5]:
avg[g]^g

1 2 2.5 2.5 3 4 2.5


We can also use the fill operator to replace a null in a given list with the corresponding value in a different list.
For example:

In [6]:
g                                 //list with some null values 1 2 0N 0N 3 4 0N

1 2 0N 0N 3 4 0N


In [7]:
show k:10 20 30 40 50 60 70       //list without null values

10 20 30 40 50 60 70


In [8]:
k^g                               //Replace nulls in g with corresponding values from k

1 2 30 40 3 4 70


<img src="../qbies.png" style="width: 50px;padding-right:5px;padding-top:5px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> To do this the lists must be of the same length!</i></p>

### The Fills Operator  
A related function is called fills. Fills replaces a null in the input list with the previous non null
value. For example:

In [9]:
show g:1 2 0N 0N 3 4 0N
fills g

1 2 0N 0N 3 4 0N


1 2 2 2 3 4 4


This is particularly useful when dealing with time associated data as we often assume the *prevailing* value is the one to use for reference in the absence of a new value. 

In some cases we might want to fill *backwards* instead of forward - do you know how you would do this? 

#### Hint - expand for help on generating a backward fill

The keyword [`reverse`](https://code.kx.com/q/ref/reverse/) will be helpful!

In [10]:
g
reverse fills reverse g

1 2 0N 0N 3 4 0N


1 2 3 3 3 4 0N


## Textual data - Symbols versus Strings 

In kdb+/q, there are two datatypes used for textual data

* Strings
* Symbols

Strings in kdb+/q are really just lists of individual characters combined together

In [11]:
show str:"hello there"

"hello there"


Strings are commonly used to represent textual data, and are the preferred datatype choice for ID columns when storing data in tables.

The other textual datatype used in kdb+/q is the Symbol type, which is commonly used for the stock ticker columns in tables.  

In [12]:
show s:`JPM`GE`IBM

`JPM`GE`IBM


As we discussed before, general lists are formed when we have lists of lists - we can see that if we are using strings to store data in a column in a table, this would create a general list.

In [13]:
show IDs:("abc123";"def567")
type IDs

"abc123"
"def567"


0h


<img src="../qbies.png" style="width: 50px;padding-right:5px;padding-top:15px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> We will go into more detail on Symbols versus Strings and their appropriate use within tables in later sections, but it's important to understand the distinction between their types and structure now. String columns are lists of lists, while Symbol columns are simple Symbol lists. </i></p>

## Commonly encountered Errors - debugging

### `'length` Error 

The most common error when working with lists is the `'length` error. This usually arises when applying pairwise operation to lists of unequal lengths.

In [14]:
show l1: til 3 
show l2: til 4

0 1 2
0 1 2 3


In [14]:
l1 + l2 

[0;31mlength[0m: [0;31mlength[0m

The first step when this error is encountered is to check the `count` of each list: 

In [15]:
count l1 
count l2 
count each (l1;l2) 

3


4


3 4


### `'type` Error

Another commonly encountered error is `'type` - in the context of lists this error may arise because you are working with a simple list (i.e. a list which is all of the same type) and your operation doesn't conform to this type.

In [15]:
l1 
type l1 
l1[2]: 2.0 //assignment fails because we can't change the inherent type of the list 

0 1 2


7h


[0;31mtype[0m: [0;31mtype[0m

In [16]:
@[l1;2;+;2] //amend list 1 at index 2 to add 2  - works because retains type 

0 1 4


In [16]:
@[l1;2;%;2] //amend list 1 at index 2 to divide by 2  - fails because resultant type is a float 

[0;31mtype[0m: [0;31mtype[0m

This does however work with general lists: 

In [17]:
show l3: (0f;1;2)
type l3 
type each l3

0f
1
2


0h


-9 -7 -7h


In [18]:
l3[2]:2.0
l3

0f
1
2f


The first step when debugging this error is to get the type of the list(s) we are working with, and the resultant types of any operations we are trying to perform:

In [19]:
type l1
type l1[2]+2   //this operation retains type   
type l1[2]%2   //this operation changes the type 

7h


-7h


-9h


In this example, a proposed solution would be the below: 

In [20]:
@[l1;2;div;2] //use the div operation to do integer division

0 1 1
